In [438]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import seaborn as sns
import plotly.express as px
import pingouin as pg
import gc
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [362]:
df_calendar = pd.read_csv("/Users/carlosito/Downloads/m5-forecasting-accuracy/calendar.csv")
df_sales_train = pd.read_csv("//Users/carlosito/Downloads/m5-forecasting-accuracy/sales_train_validation.csv")
df_sales_eval = pd.read_csv("//Users/carlosito/Downloads/m5-forecasting-accuracy/sales_train_evaluation.csv")
df_sales = pd.read_csv("/Users/carlosito/Downloads/m5-forecasting-accuracy/sell_prices.csv")

In [420]:
df_calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,has_event_1,has_event_2,is_weekend,event_1_encoder,event_2_encoder,data_chunk
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,None,NaN,None,0,0,0,0,0,0,0,0,Train
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,None,NaN,None,1,1,0,0,0,0,0,0,Train
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,None,NaN,None,1,0,1,0,0,0,0,0,Train


In [363]:
print(df_calendar.shape)
df_calendar.head()

event_1_name = [ i for i in df_calendar.event_name_1.unique() if pd.notna(i)]
event_2_name = [ i for i in df_calendar.event_name_2.unique() if pd.notna(i)]
df_calendar['has_event_1'] = df_calendar['event_name_1'].apply(lambda x: 1 if x in event_1_name else 0)
df_calendar['has_event_2'] = df_calendar['event_name_2'].apply(lambda x: 1 if x in event_2_name else 0)
df_calendar['is_weekend'] = df_calendar['weekday'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0 )

print(f"event_1 : {len(event_1_name)} | event_2: {len(event_2_name)}")
df_calendar.head()

(1969, 14)
event_1 : 30 | event_2: 4


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,has_event_1,has_event_2,is_weekend
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,0,0,1
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,0,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,0,0,0


In [364]:
df_calendar["event_type_1"] = df_calendar["event_type_1"].apply(lambda x: "None" if pd.isna(x) else x)
df_calendar["event_type_2"] = df_calendar["event_type_2"].apply(lambda x: "None" if pd.isna(x) else x)

event_1_type = df_calendar.event_type_1.unique() 
event_2_type = df_calendar.event_type_2.unique()  
event_type_label_encoder = dict(zip(event_1_type, [i for i in range(5)]))
print(f"event 1 type: {event_1_type}")
print(f"event 2 type: {event_2_type}")
print(event_type_label_encoder)

df_calendar['event_1_encoder'] = df_calendar['event_type_1'].map(lambda x: event_type_label_encoder.get(x, -1))
df_calendar['event_2_encoder']= df_calendar['event_type_2'].map(lambda x: event_type_label_encoder.get(x, -1))
df_calendar.head()

event 1 type: ['None' 'Sporting' 'Cultural' 'National' 'Religious']
event 2 type: ['None' 'Cultural' 'Religious']
{'None': 0, 'Sporting': 1, 'Cultural': 2, 'National': 3, 'Religious': 4}


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,has_event_1,has_event_2,is_weekend,event_1_encoder,event_2_encoder
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,None,NaN,None,0,0,0,0,0,1,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,None,NaN,None,0,0,0,0,0,1,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,None,NaN,None,0,0,0,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,None,NaN,None,1,1,0,0,0,0,0,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,None,NaN,None,1,0,1,0,0,0,0,0


In [365]:
df_calendar['data_chunk'] = 0
df_calendar['data_chunk'].iloc[0: 1913] = "Train"
df_calendar['data_chunk'].iloc[1913: 1913 + 28 ] = "Test"
df_calendar['data_chunk'].iloc[1913 + 28: 1913 + 28 + 28 ] = "Blind Test"

assert len(df_calendar[df_calendar['data_chunk'] == 'Train'])      == 1913
assert len(df_calendar[df_calendar['data_chunk'] == 'Test'])       == 28
assert len(df_calendar[df_calendar['data_chunk'] == 'Blind Test']) == 28

df_calendar.head()

/var/folders/tl/rdcj6pp130g_pdpr6thc58qr0000gn/T/ipykernel_2494/340171072.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,has_event_1,has_event_2,is_weekend,event_1_encoder,event_2_encoder,data_chunk
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,None,NaN,None,0,0,0,0,0,0,0,0,Train
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,None,NaN,None,1,1,0,0,0,0,0,0,Train
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,None,NaN,None,1,0,1,0,0,0,0,0,Train


In [415]:
df_calendar[df_calendar['data_chunk'] == 'Train']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,has_event_1,has_event_2,is_weekend,event_1_encoder,event_2_encoder,data_chunk
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,None,NaN,None,0,0,0,0,0,0,0,0,Train
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,None,NaN,None,1,1,0,0,0,0,0,0,Train
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,None,NaN,None,1,0,1,0,0,0,0,0,Train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,11612,Wednesday,5,4,2016,d_1909,NaN,None,NaN,None,0,0,0,0,0,0,0,0,Train
1909,2016-04-21,11612,Thursday,6,4,2016,d_1910,NaN,None,NaN,None,0,0,0,0,0,0,0,0,Train
1910,2016-04-22,11612,Friday,7,4,2016,d_1911,NaN,None,NaN,None,0,0,0,0,0,0,0,0,Train
1911,2016-04-23,11613,Saturday,1,4,2016,d_1912,NaN,None,NaN,None,0,0,0,0,0,1,0,0,Train


In [421]:
df_calendar_train = df_calendar[df_calendar['data_chunk'] == 'Train']
count_with_event = len(df_calendar_train[(df_calendar_train['has_event_1'] == 1) |  (df_calendar_train['has_event_2'] == 1)])
count_without_event = len(df_calendar_train[(df_calendar_train['has_event_1'] == 0) &  (df_calendar_train['has_event_2'] == 0)])
print(count_with_event, count_without_event)

154 1759


## Some `Exploratory Data Analysis`

In [449]:
df_eval = df_sales_train.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
merge_df = pd.merge(df_eval, df_calendar, how='left', on='d')

plot_df  = merge_df.groupby(['data_chunk', 'state_id', 'date']).aggregate({"qty": ['sum']}).reset_index()
plot_df.columns= ['data_chunk', 'state_id', 'date', 'sales_quantity']
plot_df.sort_values(by ='date', ascending=True)

,data_chunk,state_id,date,sales_quantity
0,Train,CA,2011-01-29,14195
3826,Train,WI,2011-01-29,8998
1913,Train,TX,2011-01-29,9438
1,Train,CA,2011-01-30,13805
3827,Train,WI,2011-01-30,8314
...,...,...,...,...
1911,Train,CA,2016-04-23,21834
3824,Train,TX,2016-04-23,12266
3825,Train,TX,2016-04-24,12282
1912,Train,CA,2016-04-24,23187


### Sales Volume over time by state

In [450]:
px.line(data_frame=plot_df,
        x = 'date', facet_col='state_id', color= 'state_id', width= 1200, height= 400,
        y='sales_quantity')

In [451]:
plot_df.groupby("state_id")['sales_quantity'].describe()

,count,mean,std,min,25%,50%,75%,max
state_id,,,,,,,,
CA,1913.0,14989.831155,3257.223505,5.0,12834.0,14678.0,16846.0,25224.0
TX,1913.0,9879.250392,1964.928938,2.0,8608.0,9647.0,11171.0,18733.0
WI,1913.0,9472.480920,2563.314535,1.0,7666.0,9327.0,11139.0,19669.0


### Understanding Events and Weekends Distribution

In [452]:
fig_bar = px.bar(x=["without event", "with event"], y=[count_without_event, count_with_event], width=400, text_auto='.s',
            labels={
                    "y": "Number of Days",
                    "x": " ",
                 }, title="Number of days with or without events")
fig_bar.show()

In [454]:
count_event_df= df_calendar.groupby(['data_chunk', 'event_type_1']).agg({"has_event_1": ['count']}).reset_index()
count_event_df.columns= ['data_chunk', 'event_type', 'count']
count_event_df = count_event_df.sort_values(by='count', ascending=False)
print(count_event_df.head())

fig_bar = px.bar(data_frame=count_event_df, width=1000, text_auto='.s',
                 x= "event_type",
                 y = "count", color = 'data_chunk', facet_col= "data_chunk"
                 )
fig_bar.show()


    data_chunk event_type  count
9        Train       None   1759
10       Train  Religious     52
8        Train   National     51
7        Train   Cultural     35
1   Blind Test       None     24


### Seeing correlation between `Event` and `Quantity`

In [471]:
plot_df  = merge_df.groupby(['data_chunk', 'event_type_1', 'state_id', 'cat_id', 'has_event_1', 'date']).aggregate({"qty": ['mean']}).reset_index()
plot_df.columns= ['data_chunk', 'event_type_1', 'state_id', 'cat_id', 'has_event_1', 'date', 'sales_quantity']
print(plot_df.shape)
plot_df.head()

(17217, 7)


,data_chunk,event_type_1,state_id,cat_id,has_event_1,date,sales_quantity
0,Train,Cultural,CA,FOODS,1,2011-02-14,1.127871
1,Train,Cultural,CA,FOODS,1,2011-03-17,1.098469
2,Train,Cultural,CA,FOODS,1,2011-05-05,1.112039
3,Train,Cultural,CA,FOODS,1,2011-05-08,1.291754
4,Train,Cultural,CA,FOODS,1,2011-06-19,1.371260


In [456]:
pg.pairwise_tests(dv='sales_quantity', between=['cat_id', 'has_event_1'], data=plot_df, return_desc=True).round(4)

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning:

overflow encountered in _nct_sf

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning:

overflow encountered in _nct_cdf

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/pingouin/bayesian.py:152: RuntimeWarning:

divide by zero encountered in scalar divide

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning:

overflow encountered in _nct_sf

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning:

overflow encountered in _nct_cdf

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/pingouin/bayesian.py:152: RuntimeWarning:

div

,Contrast,cat_id,A,B,mean(A),std(A),mean(B),std(B),Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,cat_id,-,FOODS,HOBBIES,1.6285,0.3873,0.5515,0.1725,False,True,192.4464,11476.0000,two-sided,0.0000,inf,3.5923
1,cat_id,-,FOODS,HOUSEHOLD,1.6285,0.3873,0.7140,0.2302,False,True,153.7802,11476.0000,two-sided,0.0000,inf,2.8706
2,cat_id,-,HOBBIES,HOUSEHOLD,0.5515,0.1725,0.7140,0.2302,False,True,-42.7987,11476.0000,two-sided,0.0000,inf,-0.7989
3,has_event_1,-,0,1,0.9693,0.5472,0.9113,0.5769,False,True,3.6064,1610.8721,two-sided,0.0003,20.598,0.1056
4,cat_id * has_event_1,FOODS,0,1,1.6336,0.3774,1.5709,0.4825,False,True,2.7194,511.5992,two-sided,0.0068,2.123,0.1619
5,cat_id * has_event_1,HOBBIES,0,1,0.5559,0.1704,0.5016,0.1883,False,True,5.9793,529.2156,two-sided,0.0000,2.576e+06,0.3154
6,cat_id * has_event_1,HOUSEHOLD,0,1,0.7186,0.2273,0.6614,0.2547,False,True,4.6708,527.3121,two-sided,0.0000,2654.651,0.2493


In [472]:
plot_df.head()

,data_chunk,event_type_1,state_id,cat_id,has_event_1,date,sales_quantity
0,Train,Cultural,CA,FOODS,1,2011-02-14,1.127871
1,Train,Cultural,CA,FOODS,1,2011-03-17,1.098469
2,Train,Cultural,CA,FOODS,1,2011-05-05,1.112039
3,Train,Cultural,CA,FOODS,1,2011-05-08,1.291754
4,Train,Cultural,CA,FOODS,1,2011-06-19,1.371260


In [473]:
plot_df.groupby(['state_id', 'cat_id', 'has_event_1'])['sales_quantity'].describe().round(2)

count  mean   std   min   25%   50%   75%   
state_id cat_id    has_event_1                                               
CA       FOODS     0            1759.0  1.75  0.36  0.88  1.51  1.70  1.97  \
                   1             154.0  1.67  0.48  0.00  1.40  1.70  2.00   
         HOBBIES   0            1759.0  0.71  0.16  0.35  0.60  0.70  0.81   
                   1             154.0  0.65  0.19  0.00  0.57  0.67  0.77   
         HOUSEHOLD 0            1759.0  0.81  0.24  0.34  0.64  0.79  0.94   
                   1             154.0  0.76  0.27  0.00  0.59  0.76  0.91   
TX       FOODS     0            1759.0  1.57  0.30  0.84  1.36  1.53  1.76   
                   1             154.0  1.57  0.43  0.00  1.36  1.58  1.85   
         HOBBIES   0            1759.0  0.49  0.12  0.20  0.40  0.48  0.57   
                   1             154.0  0.46  0.14  0.00  0.39  0.46  0.54   
         HOUSEHOLD 0            1759.0  0.73  0.21  0.26  0.58  0.72  0.86   
                   1             154.0  0.69  0.24  0.00  0.53  0.71  0.85   
WI       FOODS     0            1759.0  1.58  0.44  0.65  1.26  1.54  1.88   
                   1             154.0  1.47  0.51  0.00  1.18  1.55  1.79   
         HOBBIES   0            1759.0  0.47  0.12  0.08  0.38  0.45  0.53   
                   1             154.0  0.40  0.13  0.00  0.34  0.40  0.47   
         HOUSEHOLD 0            1759.0  0.62  0.20  0.11  0.48  0.60  0.74   
                   1             154.0  0.54  0.21  0.00  0.40  0.54  0.67   

                                 max  
state_id cat_id    has_event_1        
CA       FOODS     0            2.87  
                   1            2.62  
         HOBBIES   0            1.19  
                   1            1.19  
         HOUSEHOLD 0            1.57  
                   1            1.30  
TX       FOODS     0            3.32  
                   1            2.36  
         HOBBIES   0            0.90  
                   1            0.81  
         HOUSEHOLD 0            1.33  
                   1            1.21  
WI       FOODS     0            3.33  
                   1            2.76  
         HOBBIES   0            0.94  
                   1            0.72  
         HOUSEHOLD 0            1.40  
                   1            1.06

In [458]:
px.box(data_frame=plot_df,
        x = 'has_event_1', facet_col='cat_id', color= 'cat_id', width = 800, height=1000, facet_row='state_id',
        y='sales_quantity').update_traces(boxmean= True)

### Seeing correlation between `Weekend` and `Quantity`

In [474]:
plot_df  = merge_df.groupby(['data_chunk' , 'state_id', 'cat_id', 'is_weekend', 'date']).aggregate({"qty": ['mean']}).reset_index()
plot_df.columns= ['data_chunk' , 'state_id', 'cat_id', 'is_weekend', 'date', 'sales_quantity']
print(plot_df.shape)
plot_df.head()

(17217, 6)


,data_chunk,state_id,cat_id,is_weekend,date,sales_quantity
0,Train,CA,FOODS,0,2011-01-31,1.208072
1,Train,CA,FOODS,0,2011-02-01,1.368128
2,Train,CA,FOODS,0,2011-02-02,1.248782
3,Train,CA,FOODS,0,2011-02-03,1.436326
4,Train,CA,FOODS,0,2011-02-04,1.566632


In [470]:
pg.pairwise_tests(dv='sales_quantity', between=[ 'is_weekend'], data=plot_df, return_desc=True).round(3)

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning:

overflow encountered in _nct_sf

/Users/carlosito/Documents/Virtual_Environment/persona/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning:

overflow encountered in _nct_cdf



,Contrast,A,B,mean(A),std(A),mean(B),std(B),Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,is_weekend,0,1,0.887,0.492,1.158,0.632,False,True,-26.989,7444.283,two-sided,0.0,1.129e+153,-0.505


In [468]:
px.box(data_frame=plot_df,
        x = 'is_weekend', width=400,
        y='sales_quantity').update_traces(boxmean= True)

In [475]:
plot_df.groupby(['state_id', 'cat_id', 'is_weekend'])['sales_quantity'].describe().round(2)

count  mean   std  min   25%   50%   75%   max
state_id cat_id    is_weekend                                                 
CA       FOODS     0           1365.0  1.60  0.27  0.0  1.44  1.62  1.76  2.62
                   1            548.0  2.12  0.32  0.0  1.96  2.15  2.35  2.87
         HOBBIES   0           1365.0  0.66  0.14  0.0  0.57  0.66  0.75  1.19
                   1            548.0  0.82  0.16  0.0  0.71  0.81  0.93  1.19
         HOUSEHOLD 0           1365.0  0.72  0.17  0.0  0.57  0.74  0.84  1.35
                   1            548.0  1.02  0.24  0.0  0.84  1.07  1.19  1.57
TX       FOODS     0           1365.0  1.45  0.24  0.0  1.30  1.45  1.59  3.32
                   1            548.0  1.87  0.26  0.0  1.71  1.88  2.05  2.60
         HOBBIES   0           1365.0  0.45  0.10  0.0  0.38  0.45  0.53  0.82
                   1            548.0  0.58  0.11  0.0  0.49  0.57  0.67  0.90
         HOUSEHOLD 0           1365.0  0.66  0.17  0.0  0.53  0.68  0.78  1.21
                   1            548.0  0.89  0.21  0.0  0.71  0.93  1.05  1.33
WI       FOODS     0           1365.0  1.46  0.38  0.0  1.19  1.44  1.73  2.71
                   1            548.0  1.84  0.47  0.0  1.52  1.86  2.19  3.33
         HOBBIES   0           1365.0  0.43  0.10  0.0  0.37  0.42  0.48  0.87
                   1            548.0  0.54  0.13  0.0  0.45  0.54  0.62  0.94
         HOUSEHOLD 0           1365.0  0.56  0.17  0.0  0.44  0.55  0.67  1.35
                   1            548.0  0.74  0.21  0.0  0.58  0.74  0.88  1.40

In [467]:
px.box(data_frame=plot_df,
        x = 'is_weekend', facet_col='cat_id', color= 'cat_id', width = 800, height=1000, facet_row='state_id',
        y='sales_quantity').update_traces(boxmean= True)

## Training Data

In [277]:
sales_day_train = [f"d_{i}" for i in range(1, 1914)]
sales_day_test = [f"d_{i}" for i in range(1914, 1914 + 28)]
sales_day_blind_test = [f"d_{i}" for i in range(1914 + 28, 1914 + 28 +28)]

In [31]:
select_sales_col = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id']
select_sales_col.extend(sales_day_train)
print(f"number of train columns: {len(select_sales_col)}")
df_sales_eval[select_sales_col].head()

number of train columns: 1918


,id,item_id,dept_id,cat_id,store_id,d_1,d_2,d_3,d_4,d_5,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


### Method 1: use only 1-year before

In [61]:
sales_day_train_one_year = sales_day_train[-365:]
print(len(sales_day_train_one_year))



select_sales_col = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
select_sales_col.extend(sales_day_train_one_year)
print(f"number of train columns: {len(select_sales_col)}")
df_train_one_year = df_sales_eval[select_sales_col]
df_train_one_year.head()

365
number of train columns: 371


,id,item_id,dept_id,cat_id,store_id,state_id,d_1549,d_1550,d_1551,d_1552,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,3,0,2,1,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,2,1,2,...,2,1,1,0,1,1,2,2,2,4


In [93]:
cut_off_day = 30
train_day = sales_day_train_one_year[: -cut_off_day]
test_day = sales_day_train_one_year[-cut_off_day: ]

print(train_day)
print(test_day)

['d_1549', 'd_1550', 'd_1551', 'd_1552', 'd_1553', 'd_1554', 'd_1555', 'd_1556', 'd_1557', 'd_1558', 'd_1559', 'd_1560', 'd_1561', 'd_1562', 'd_1563', 'd_1564', 'd_1565', 'd_1566', 'd_1567', 'd_1568', 'd_1569', 'd_1570', 'd_1571', 'd_1572', 'd_1573', 'd_1574', 'd_1575', 'd_1576', 'd_1577', 'd_1578', 'd_1579', 'd_1580', 'd_1581', 'd_1582', 'd_1583', 'd_1584', 'd_1585', 'd_1586', 'd_1587', 'd_1588', 'd_1589', 'd_1590', 'd_1591', 'd_1592', 'd_1593', 'd_1594', 'd_1595', 'd_1596', 'd_1597', 'd_1598', 'd_1599', 'd_1600', 'd_1601', 'd_1602', 'd_1603', 'd_1604', 'd_1605', 'd_1606', 'd_1607', 'd_1608', 'd_1609', 'd_1610', 'd_1611', 'd_1612', 'd_1613', 'd_1614', 'd_1615', 'd_1616', 'd_1617', 'd_1618', 'd_1619', 'd_1620', 'd_1621', 'd_1622', 'd_1623', 'd_1624', 'd_1625', 'd_1626', 'd_1627', 'd_1628', 'd_1629', 'd_1630', 'd_1631', 'd_1632', 'd_1633', 'd_1634', 'd_1635', 'd_1636', 'd_1637', 'd_1638', 'd_1639', 'd_1640', 'd_1641', 'd_1642', 'd_1643', 'd_1644', 'd_1645', 'd_1646', 'd_1647', 'd_1648',

In [103]:
len(train_day) * 30490

10214150

#### Melting the dataframe

In [62]:
df_train_one_year_new  = df_train_one_year.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
df_train_one_year_new.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,qty
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,3
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0


In [63]:
df_train_one_year_new.shape

(11128850, 8)

In [64]:
df_train_one_year_new = pd.merge(df_train_one_year_new, df_calendar, how='left', on='d')
print(df_train_one_year_new.shape)
df_train_one_year_new.head()

(11128850, 27)


,id,item_id,dept_id,cat_id,store_id,state_id,d,qty,date,wm_yr_wk,...,event_type_2,snap_CA,snap_TX,snap_WI,has_event_1,has_event_2,is_weekend,event_1_encoder,event_2_encoder,data_chunk
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,2015-04-26,11513,...,None,0,0,0,0,0,1,0,0,Train
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,2015-04-26,11513,...,None,0,0,0,0,0,1,0,0,Train
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,2015-04-26,11513,...,None,0,0,0,0,0,1,0,0,Train
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,3,2015-04-26,11513,...,None,0,0,0,0,0,1,0,0,Train
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,2015-04-26,11513,...,None,0,0,0,0,0,1,0,0,Train


In [72]:
dummy_df = pd.get_dummies(data=df_train_one_year_new, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
print(dummy_df.shape)
dummy_df.head()

(11128850, 46)


,id,item_id,d,qty,date,wm_yr_wk,weekday,wday,month,year,...,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,d_1549,0,2015-04-26,11513,Sunday,2,4,2015,...,False,False,False,False,False,False,False,True,False,False
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,d_1549,0,2015-04-26,11513,Sunday,2,4,2015,...,False,False,False,False,False,False,False,True,False,False
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,d_1549,0,2015-04-26,11513,Sunday,2,4,2015,...,False,False,False,False,False,False,False,True,False,False
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,d_1549,3,2015-04-26,11513,Sunday,2,4,2015,...,False,False,False,False,False,False,False,True,False,False
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,d_1549,0,2015-04-26,11513,Sunday,2,4,2015,...,False,False,False,False,False,False,False,True,False,False


In [316]:
target_col = 'qty'
exclude_cols = ['id', 'item_id', 'd', 'date', 'wm_yr_wk', 'qty', 'weekday', 'event_name_1', 'event_name_2',  'event_type_1', 'event_type_2', 'data_chunk' ]
feature_cols = [col for col in dummy_df.columns if col not in exclude_cols]

In [318]:
len(feature_cols)

34

In [319]:
X_train.shape

(10214150, 34)

In [104]:
df_train  = dummy_df[dummy_df['d'].isin(train_day)]
X_train = np.array(df_train[feature_cols])
y_train =  np.array(df_train[target_col])

In [111]:
df_test  = dummy_df[dummy_df['d'].isin(test_day)]
X_test = np.array(df_test[feature_cols])
y_test =  np.array(df_test[target_col])

In [113]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10214150, 34), (10214150,), (914700, 34), (914700,))

In [108]:
X_train[0]

array([2, 4, 2015, 0, 0, 0, 0, 0, 1, 0, 0, False, False, False, True,
       False, False, False, False, True, False, True, False, False, False,
       False, False, False, False, False, False, True, False, False],
      dtype=object)

#### Training Model

In [174]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor


In [175]:
reg = LinearRegression().fit(X_train, y_train)


In [118]:
prediction = reg.predict(X_test)

### Evaluate the result

In [165]:
mean_squared_error(y_true = y_test,
                   y_pred=prediction,
                   squared=False)

3.5857502340639713

In [185]:
df_train_one_year_new['cat_id'].unique()

array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object)

In [187]:
state = ['CA', 'TX', 'WI']
cat = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
for s in state:
    for c in cat:
        filter_df = result_df[(result_df['state_id'] == s) & (result_df['cat_id'] == c) ]
        rmse = mean_squared_error(y_true = filter_df.y_true,
                    y_pred=filter_df.prediction,
                    squared=False)
        print(f"RMSE for {s} - {c}: {rmse}")

RMSE for CA - HOBBIES: 2.3890827420725276
RMSE for CA - HOUSEHOLD: 2.291699662640033
RMSE for CA - FOODS: 4.584699334109162
RMSE for TX - HOBBIES: 1.8222483012470125
RMSE for TX - HOUSEHOLD: 1.8988430885565188
RMSE for TX - FOODS: 4.468403466293551
RMSE for WI - HOBBIES: 1.6840311323430694
RMSE for WI - HOUSEHOLD: 2.050868225159104
RMSE for WI - FOODS: 5.170850220376382


In [478]:
result_df = pd.DataFrame({'date': df_test['date'],
                          'cat_id': df_train_one_year_new[df_train_one_year_new['d'].isin(test_day)]['cat_id'],
                          'state_id' : df_train_one_year_new[df_train_one_year_new['d'].isin(test_day)]['state_id'],
              'y_true': y_test,
             'prediction': prediction})
group_result_df= result_df.groupby(['date', 'state_id', 'cat_id']).agg({'y_true': ['mean'],
                               'prediction' : ['mean']}).reset_index()
group_result_df.columns = ['date', 'state_id', 'cat_id','y_true_mean', 'prediction_mean']
result_df.head()

,date,cat_id,state_id,y_true,prediction
10214150,2016-03-26,HOBBIES,CA,1,1.263559
10214151,2016-03-26,HOBBIES,CA,1,1.263559
10214152,2016-03-26,HOBBIES,CA,1,1.263559
10214153,2016-03-26,HOBBIES,CA,6,1.263559
10214154,2016-03-26,HOBBIES,CA,0,1.263559


In [479]:
result_df.tail()

,date,cat_id,state_id,y_true,prediction
11128845,2016-04-24,FOODS,WI,1,2.245354
11128846,2016-04-24,FOODS,WI,0,2.245354
11128847,2016-04-24,FOODS,WI,0,2.245354
11128848,2016-04-24,FOODS,WI,3,2.245354
11128849,2016-04-24,FOODS,WI,0,2.245354


In [343]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
             labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 800,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

In [344]:
state = ['CA', 'TX', 'WI']
for s in state:
    filter_df = result_df[result_df['state_id'] == s]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {s}: {rmse}")
    
print()

cat = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
for c in cat:
    filter_df = result_df[result_df['cat_id'] == c]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {c}: {rmse}")
    
print()
    
    
for s in state:
    for c in cat:
        filter_df = result_df[(result_df['state_id'] == s) & (result_df['cat_id'] == c) ]
        rmse = mean_squared_error(y_true = filter_df.y_true,
                    y_pred=filter_df.prediction,
                    squared=False)
        print(f"RMSE for {s} - {c}: {rmse}")

RMSE for CA: 3.5731846899948843
RMSE for TX: 3.3561546910185696
RMSE for WI: 3.8172472265366415

RMSE for HOBBIES: 2.032252803471312
RMSE for HOUSEHOLD: 2.1081400814641356
RMSE for FOODS: 4.734879476334916

RMSE for CA - HOBBIES: 2.3890827420725276
RMSE for CA - HOUSEHOLD: 2.291699662640033
RMSE for CA - FOODS: 4.584699334109162
RMSE for TX - HOBBIES: 1.8222483012470125
RMSE for TX - HOUSEHOLD: 1.8988430885565188
RMSE for TX - FOODS: 4.468403466293551
RMSE for WI - HOBBIES: 1.6840311323430694
RMSE for WI - HOUSEHOLD: 2.050868225159104
RMSE for WI - FOODS: 5.170850220376382


In [158]:
mean_squared_error(y_true = result_df.y_true_mean,
                   y_pred=result_df.prediction_mean,
                   squared=False)

0.22336650983859302

In [162]:
# Plotly line plot
fig = px.line(result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
              facet_col='cat_id',facet_row ='state_id', labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 1200, height=1000,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

### Now predict on the `Evaluation` set

In [215]:
select_sales_col = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
select_sales_col.extend(sales_day_test)
print(f"number of train columns: {len(select_sales_col)}")
df_eval_one_year = df_sales_eval[select_sales_col]
df_eval_one_year  = df_eval_one_year.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
print(df_eval_one_year.shape)
assert len(df_eval_one_year) == 30490 * 28

df_eval_one_year_new = pd.merge(df_eval_one_year, df_calendar, how='left', on='d')
print(df_eval_one_year_new.shape)

df_eval_one_year_new = pd.get_dummies(data=df_eval_one_year_new, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
print(df_eval_one_year_new.shape)
df_eval_one_year_new.head()

X = np.array(df_eval_one_year_new[feature_cols])
y = np.array(df_eval_one_year_new[target_col])

number of train columns: 34
(853720, 8)
(853720, 27)
(853720, 46)


In [212]:
eval_prediction = reg.predict(X )
print(mean_squared_error(y_true = y,
                   y_pred=eval_prediction,
                   squared=False))

3.549433248006835


In [340]:
result_df = pd.DataFrame({'date': df_eval_one_year_new['date'],
                          'cat_id': df_eval_one_year[df_eval_one_year['d'].isin(sales_day_test)]['cat_id'],
                          'state_id' : df_eval_one_year[df_eval_one_year['d'].isin(sales_day_test)]['state_id'],
              'y_true': y,
             'prediction': eval_prediction})
group_result_df= result_df.groupby(['date', 'state_id', 'cat_id']).agg({'y_true': ['mean'],
                               'prediction' : ['mean']}).reset_index()
group_result_df.columns = ['date', 'state_id', 'cat_id','y_true_mean', 'prediction_mean']
result_df.head()

,date,cat_id,state_id,y_true,prediction
0,2016-04-25,HOBBIES,CA,0,0.904401
1,2016-04-25,HOBBIES,CA,0,0.904401
2,2016-04-25,HOBBIES,CA,0,0.904401
3,2016-04-25,HOBBIES,CA,0,0.904401
4,2016-04-25,HOBBIES,CA,1,0.904401


In [480]:
state = ['CA', 'TX', 'WI']
for s in state:
    filter_df = result_df[result_df['state_id'] == s]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {s}: {rmse}")
    
print()

cat = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
for c in cat:
    filter_df = result_df[result_df['cat_id'] == c]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {c}: {rmse}")
    
print()
    
    
for s in state:
    for c in cat:
        filter_df = result_df[(result_df['state_id'] == s) & (result_df['cat_id'] == c) ]
        rmse = mean_squared_error(y_true = filter_df.y_true,
                    y_pred=filter_df.prediction,
                    squared=False)
        print(f"RMSE for {s} - {c}: {rmse}")

RMSE for CA: 3.5731846899948843
RMSE for TX: 3.3561546910185696
RMSE for WI: 3.8172472265366415

RMSE for HOBBIES: 2.032252803471312
RMSE for HOUSEHOLD: 2.1081400814641356
RMSE for FOODS: 4.734879476334916

RMSE for CA - HOBBIES: 2.3890827420725276
RMSE for CA - HOUSEHOLD: 2.291699662640033
RMSE for CA - FOODS: 4.584699334109162
RMSE for TX - HOBBIES: 1.8222483012470125
RMSE for TX - HOUSEHOLD: 1.8988430885565188
RMSE for TX - FOODS: 4.468403466293551
RMSE for WI - HOBBIES: 1.6840311323430694
RMSE for WI - HOUSEHOLD: 2.050868225159104
RMSE for WI - FOODS: 5.170850220376382


In [482]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
             labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 800,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

In [481]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
              facet_col='cat_id',facet_row ='state_id', labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 1200, height=1000,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

### Method 2: Add windows

In [358]:
gc.collect()

6448

In [305]:
df_train_one_year_window = df_train_one_year_new.copy(deep=True)

#### Adding `Lag` & `Moving average`

In [307]:
lags=[3 ,7 ,14 ,28]
for i in tqdm(lags):
    df_train_one_year_window['lag_'+str(i)]=df_train_one_year_window.groupby(['id'])['qty'].shift(i)
    
#Replacing 'NaN' in 'lags' features with 0
lags_name=[f"lag_{str(l)}" for l in lags]
for i in tqdm(lags_name):
    df_train_one_year_window[i]=df_train_one_year_window[i].fillna(0)

100%|██████████| 4/4 [00:00<00:00, 27.98it/s]


In [308]:
df_train_one_year_window[df_train_one_year_window.id == 'FOODS_3_827_WI_3_evaluation'].head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,qty,date,wm_yr_wk,...,has_event_1,has_event_2,is_weekend,event_1_encoder,event_2_encoder,data_chunk,lag_3,lag_7,lag_14,lag_28
30489,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1549,0,2015-04-26,11513,...,0,0,1,0,0,Train,0.0,0.0,0.0,0.0
60979,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1550,2,2015-04-27,11513,...,0,0,0,0,0,Train,0.0,0.0,0.0,0.0
91469,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1551,4,2015-04-28,11513,...,0,0,0,0,0,Train,0.0,0.0,0.0,0.0
121959,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1552,0,2015-04-29,11513,...,0,0,0,0,0,Train,0.0,0.0,0.0,0.0
152449,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1553,0,2015-04-30,11513,...,0,0,0,0,0,Train,2.0,0.0,0.0,0.0
182939,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1554,0,2015-05-01,11513,...,0,0,0,0,0,Train,4.0,0.0,0.0,0.0
213429,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1555,5,2015-05-02,11514,...,0,0,1,0,0,Train,0.0,0.0,0.0,0.0
243919,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1556,10,2015-05-03,11514,...,0,0,1,0,0,Train,0.0,0.0,0.0,0.0
274409,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1557,5,2015-05-04,11514,...,0,0,0,0,0,Train,0.0,2.0,0.0,0.0
304899,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1558,5,2015-05-05,11514,...,1,0,0,2,0,Train,5.0,4.0,0.0,0.0


In [477]:
feature_cols_window

['wday',
 'month',
 'year',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'has_event_1',
 'has_event_2',
 'is_weekend',
 'event_1_encoder',
 'event_2_encoder',
 'lag_3',
 'lag_7',
 'lag_14',
 'lag_28',
 'rolling_median_3',
 'rolling_median_7',
 'rolling_median_14',
 'rolling_median_28',
 'dept_id_FOODS_1',
 'dept_id_FOODS_2',
 'dept_id_FOODS_3',
 'dept_id_HOBBIES_1',
 'dept_id_HOBBIES_2',
 'dept_id_HOUSEHOLD_1',
 'dept_id_HOUSEHOLD_2',
 'cat_id_FOODS',
 'cat_id_HOBBIES',
 'cat_id_HOUSEHOLD',
 'store_id_CA_1',
 'store_id_CA_2',
 'store_id_CA_3',
 'store_id_CA_4',
 'store_id_TX_1',
 'store_id_TX_2',
 'store_id_TX_3',
 'store_id_WI_1',
 'store_id_WI_2',
 'store_id_WI_3',
 'state_id_CA',
 'state_id_TX',
 'state_id_WI']

In [309]:

#Rolling-Mean
window=[3 ,7 ,14 ,28]
for i in tqdm(window):
  df_train_one_year_window['rolling_median_'+str(i)]=df_train_one_year_window.groupby(['id'])['qty'].transform(lambda s: s.rolling(i, center=False).median())
  
#Replacing 'NaN' in 'rolling_ mean' features with 0
window_name = [f"rolling_median_{str(w)}" for w in window]
for i in window_name:
  df_train_one_year_window[i]=df_train_one_year_window[i].fillna(0)
  

100%|██████████| 4/4 [00:32<00:00,  8.02s/it]


In [311]:
df_train_one_year_window.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,qty,date,wm_yr_wk,...,event_2_encoder,data_chunk,lag_3,lag_7,lag_14,lag_28,rolling_median_3,rolling_median_7,rolling_median_14,rolling_median_28
11128845,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1,2016-04-24,11613,...,0,Train,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11128846,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,0,Train,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11128847,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,0,Train,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.5
11128848,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3,2016-04-24,11613,...,0,Train,0.0,1.0,2.0,4.0,3.0,1.0,0.5,1.0
11128849,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,0,Train,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0


In [320]:
dummy_df_window = pd.get_dummies(data=df_train_one_year_window, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
print(dummy_df_window.shape)
dummy_df_window.head()

target_col = 'qty'
exclude_cols_window = ['id', 'item_id', 'd', 'date', 'wm_yr_wk', 'qty', 'weekday', 'event_name_1', 'event_name_2',  'event_type_1', 'event_type_2', 'data_chunk' ]
feature_cols_window = [col for col in dummy_df_window.columns if col not in exclude_cols]



(11128850, 54)


In [323]:
len(feature_cols) , len(feature_cols_window) 

(34, 42)

#### Preparing `Training Data`

In [326]:
df_train_window  = dummy_df_window[dummy_df_window['d'].isin(train_day)]
X_train_window = np.array(df_train_window[feature_cols_window])
y_train_window =  np.array(df_train_window[target_col])


df_test_window  = dummy_df_window[dummy_df_window['d'].isin(test_day)]
X_test_window = np.array(df_test_window[feature_cols_window])
y_test_window =  np.array(df_test_window[target_col])

print(X_train_window.shape, y_train_window.shape, X_test_window.shape, y_test_window.shape)

(10214150, 42) (10214150,) (914700, 42) (914700,)


#### Training Model

In [327]:
reg_window = LinearRegression().fit(X_train_window, y_train_window)
prediction_window = reg_window.predict(X_test_window)

#### Evaluate the result

In [328]:
mean_squared_error(y_true = y_test_window,
                   y_pred=prediction_window,
                   squared=False)

1.7258427211906644

##### Prediction on the `put-aside` data

In [352]:
result_df = pd.DataFrame({'date': df_test['date'],
                          'cat_id': df_train_one_year_new[df_train_one_year_new['d'].isin(test_day)]['cat_id'],
                          'state_id' : df_train_one_year_new[df_train_one_year_new['d'].isin(test_day)]['state_id'],
              'y_true': y_test_window,
             'prediction': prediction_window})
group_result_df= result_df.groupby(['date', 'state_id', 'cat_id']).agg({'y_true': ['mean'],
                               'prediction' : ['mean']}).reset_index()
group_result_df.columns = ['date', 'state_id', 'cat_id','y_true_mean', 'prediction_mean']
result_df.head()

,date,cat_id,state_id,y_true,prediction
10214150,2016-03-26,HOBBIES,CA,1,1.384218
10214151,2016-03-26,HOBBIES,CA,1,1.109772
10214152,2016-03-26,HOBBIES,CA,1,1.109772
10214153,2016-03-26,HOBBIES,CA,6,4.638342
10214154,2016-03-26,HOBBIES,CA,0,0.624466


In [353]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
             labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 800,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

In [354]:
state = ['CA', 'TX', 'WI']
for s in state:
    filter_df = result_df[result_df['state_id'] == s]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {s}: {rmse}")
    
print()

cat = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
for c in cat:
    filter_df = result_df[result_df['cat_id'] == c]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {c}: {rmse}")
    
print()
    
    
for s in state:
    for c in cat:
        filter_df = result_df[(result_df['state_id'] == s) & (result_df['cat_id'] == c) ]
        rmse = mean_squared_error(y_true = filter_df.y_true,
                    y_pred=filter_df.prediction,
                    squared=False)
        print(f"RMSE for {s} - {c}: {rmse}")

RMSE for CA: 1.7888227817829057
RMSE for TX: 1.503397150752876
RMSE for WI: 1.8443765184735879

RMSE for HOBBIES: 1.5025599520350181
RMSE for HOUSEHOLD: 1.1924526690523085
RMSE for FOODS: 2.0966852403955123

RMSE for CA - HOBBIES: 1.7394464170668535
RMSE for CA - HOUSEHOLD: 1.29036872289735
RMSE for CA - FOODS: 2.094437195438546
RMSE for TX - HOBBIES: 1.3601213462542678
RMSE for TX - HOUSEHOLD: 1.1005481987731047
RMSE for TX - FOODS: 1.7848850024814893
RMSE for WI - HOBBIES: 1.281194561545115
RMSE for WI - HOUSEHOLD: 1.1439127154500055
RMSE for WI - FOODS: 2.370427380180482


In [351]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
              facet_col='cat_id',facet_row ='state_id', labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 1200, height=1000,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

##### Prediction on the `first 28-day` - eval data

In [398]:
df_eval_one_year_new.shape

(853720, 46)

In [399]:
df_test[df_test['d'].isin([f"d_{i}" for i in range(1914-28, 1914)])].shape

(853720, 46)

In [406]:
df_eval_one_year_new_window = df_eval_one_year_new.copy(deep=True)
df_eval_one_year_new_window = pd.concat([df_test[df_test['d'].isin([f"d_{i}" for i in range(1914-28, 1914)])], 
                                         df_eval_one_year_new_window])

lags=[3 ,7 ,14 ,28]
for i in tqdm(lags):
    df_eval_one_year_new_window['lag_'+str(i)]=df_eval_one_year_new_window.groupby(['id'])['qty'].shift(i)
    
#Replacing 'NaN' in 'lags' features with 0
lags_name=[f"lag_{str(l)}" for l in lags]
for i in tqdm(lags_name):
    df_eval_one_year_new_window[i]=df_eval_one_year_new_window[i].fillna(0)
    

#Rolling-Mean
window=[3 ,7 ,14 ,28]
for i in tqdm(window):
  df_eval_one_year_new_window['rolling_median_'+str(i)]=df_eval_one_year_new_window.groupby(['id'])['qty'].transform(lambda s: s.rolling(i, center=False).median())
  
#Replacing 'NaN' in 'rolling_ mean' features with 0
window_name = [f"rolling_median_{str(w)}" for w in window]
for i in window_name:
  df_eval_one_year_new_window[i]=df_eval_one_year_new_window[i].fillna(0)


df_eval_one_year_new_window  = df_eval_one_year_new_window[~df_eval_one_year_new_window['d'].isin([f"d_{i}" for i in range(1914-28, 1914)])]
print(df_eval_one_year_new_window.shape)

100%|██████████| 4/4 [00:15<00:00,  3.79s/it]


(853720, 54)


In [407]:
df_eval_one_year_new_window.head()

,id,item_id,d,qty,date,wm_yr_wk,weekday,wday,month,year,...,state_id_TX,state_id_WI,lag_3,lag_7,lag_14,lag_28,rolling_median_3,rolling_median_7,rolling_median_14,rolling_median_28
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,d_1914,0,2016-04-25,11613,Monday,3,4,2016,...,False,False,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,d_1914,0,2016-04-25,11613,Monday,3,4,2016,...,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,d_1914,0,2016-04-25,11613,Monday,3,4,2016,...,False,False,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,d_1914,0,2016-04-25,11613,Monday,3,4,2016,...,False,False,3.0,4.0,2.0,0.0,2.0,1.0,1.0,1.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,d_1914,1,2016-04-25,11613,Monday,3,4,2016,...,False,False,2.0,0.0,2.0,1.0,2.0,2.0,1.0,1.0


In [408]:
X_window = np.array(df_eval_one_year_new_window[feature_cols_window])
y_window = np.array(df_eval_one_year_new_window[target_col])

print(X_window.shape, y_window.shape)

(853720, 42) (853720,)


In [409]:
eval_prediction_window = reg_window.predict(X_window)
mean_squared_error(y_true = y_window,
                   y_pred=eval_prediction_window,
                   squared=False)

1.7228007065715902

In [410]:
result_df = pd.DataFrame({'date': df_eval_one_year_new['date'],
                          'cat_id': df_eval_one_year[df_eval_one_year['d'].isin(sales_day_test)]['cat_id'],
                          'state_id' : df_eval_one_year[df_eval_one_year['d'].isin(sales_day_test)]['state_id'],
              'y_true': y_window,
             'prediction': eval_prediction_window})
group_result_df= result_df.groupby(['date', 'state_id', 'cat_id']).agg({'y_true': ['mean'],
                               'prediction' : ['mean']}).reset_index()
group_result_df.columns = ['date', 'state_id', 'cat_id','y_true_mean', 'prediction_mean']
result_df.head()

,date,cat_id,state_id,y_true,prediction
0,2016-04-25,HOBBIES,CA,0,1.023680
1,2016-04-25,HOBBIES,CA,0,0.074059
2,2016-04-25,HOBBIES,CA,0,0.976894
3,2016-04-25,HOBBIES,CA,0,1.911273
4,2016-04-25,HOBBIES,CA,1,1.941422


In [411]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
             labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 800,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

In [485]:
gc.collect()

0

In [412]:
state = ['CA', 'TX', 'WI']
for s in state:
    filter_df = result_df[result_df['state_id'] == s]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {s}: {rmse}")
    
print()

cat = ['HOBBIES', 'HOUSEHOLD', 'FOODS']
for c in cat:
    filter_df = result_df[result_df['cat_id'] == c]
    rmse = mean_squared_error(y_true = filter_df.y_true,
                   y_pred=filter_df.prediction,
                   squared=False)
    print(f"RMSE for {c}: {rmse}")
    
print()
    
    
for s in state:
    for c in cat:
        filter_df = result_df[(result_df['state_id'] == s) & (result_df['cat_id'] == c) ]
        rmse = mean_squared_error(y_true = filter_df.y_true,
                    y_pred=filter_df.prediction,
                    squared=False)
        print(f"RMSE for {s} - {c}: {rmse}")

RMSE for CA: 1.7344148318574542
RMSE for TX: 1.527905995188467
RMSE for WI: 1.883627114493794

RMSE for HOBBIES: 1.4797627895163914
RMSE for HOUSEHOLD: 1.1486787631923754
RMSE for FOODS: 2.1154736801263527

RMSE for CA - HOBBIES: 1.760788155013692
RMSE for CA - HOUSEHOLD: 1.2403852760488185
RMSE for CA - FOODS: 2.010654707521566
RMSE for TX - HOBBIES: 1.3125357669578281
RMSE for TX - HOUSEHOLD: 1.0698350042377438
RMSE for TX - FOODS: 1.8552687971550608
RMSE for WI - HOBBIES: 1.2010036089866005
RMSE for WI - HOUSEHOLD: 1.096473995596752
RMSE for WI - FOODS: 2.4667990441371086


In [414]:
# Plotly line plot
fig = px.line(group_result_df, x='date', y=['y_true_mean', 'prediction_mean'], 
              facet_col='cat_id',facet_row ='state_id', labels={'value': 'Mean Value', 'variable': 'Metric'}, width = 1200, height=1000,
              title=f'True Mean vs Prediction Mean Over Time')

# Show the plot
fig.show()

In [105]:
gc.collect()

266